In [1]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

data = pd.read_csv("data.tsv", skipinitialspace = True, sep = "\t")

data.head()
data.tail(10)
data.drop(data.tail(5).index, inplace=True)
data.tail(10)

,Kooli ID,Kooli nimi,Maakond,Asutuse tüüp,Seadmete nimekiri,Toetus 2014,Toetus 2015,Toetus 2016,Toetus 2017,Toetus 2018,Toetus 2019,Toetus 2020,Toetus 2021
682,535,Viluste Põhikool,Põlva maakond,põhikool,"Original Prusa i3 kit with LCD 1.75mm (2 tk), ...",NaN,"1,296.00 €",NaN,NaN,NaN,NaN,NaN,NaN
683,42931,Vivere Kool,Harju maakond,põhikool,"mTiny avastajakomplekt (2tk), LEGO Education W...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,189.46 €"
684,233,Voore Põhikool,Jõgeva maakond,põhikool,"LEGO Mindstorms EV3 baaskomplekt (5 tk), EV3 l...",NaN,"1,750.32 €","1,116.35 €",NaN,NaN,NaN,"2,174.76 €",NaN
685,531,Võru Kesklinna Kool,Võru maakond,põhikool,"LEGO Mindstorms EV3 baaskomplekt (6 tk), EV3 l...",NaN,"3,071.89 €",NaN,NaN,"3,408.90 €",NaN,NaN,NaN
686,7131,Võru Kreutzwaldi Kool,Võru maakond,põhikool,"LEGO Mindstorms EV3 baaskomplektid (12 tk), li...","2,723.76 €","2,916.07 €","1,403.64 €",NaN,"3,191.92 €",NaN,"2,776.03 €",NaN
687,660,Võsu Kool,Lääne-Viru maakond,põhikool,"Snapmaker 3-1 (3D Printing, Laser Engraver, CN...",NaN,NaN,NaN,NaN,NaN,"1,933.92 €","1,809.37 €",NaN
688,4811,Vääna-Jõesuu Kool,Harju maakond,põhikool,"LEGO WeDo baaskomplektid 3 (tk), lisakomplekt ...",948.47 €,NaN,"1,021.75 €",NaN,NaN,NaN,NaN,NaN
689,399,Väätsa Põhikool,Järva maakond,põhikool,"LEGO Mindstorms EV3 baaskomplekt (4 tk), värv...",752.94 €,"1,488.64 €","1,796.10 €",NaN,NaN,NaN,NaN,NaN
690,380,Ääsmäe Põhikool,Harju maakond,põhikool,"LEGO Mindstorms EV3 baaskomplektid (5 tk), lis...","1,209.60 €","2,312.93 €",NaN,NaN,NaN,"1,436.40 €",NaN,NaN
691,249,Kohtla-Järve Täiskasvanute Gümnaasium,Ida-Viru maakond,täiskasvanute gümnaasium,mBot robootikakomplekt (5tk); Ülim robootikako...,NaN,NaN,NaN,NaN,NaN,NaN,944.87 €,NaN


In [2]:
#Translations of column names:

#Kooli ID - school's ID
#Maakond - county
#Asutuse tüüp - Type of school (high school, middle school, vocational school etc)
#Seadmete nimekiri - list of devices
#Toetus - funding

In [3]:
#basic check for possible typos in some columns
print("Maakonnad:")
data['Maakond'].value_counts()
print("Asutused:")
data['Asutuse tüüp'].value_counts()

Maakonnad:


Harju maakond         201
Tartu maakond          96
Ida-Viru maakond       64
Pärnu maakond          60
Lääne-Viru maakond     41
Viljandi maakond       33
Rapla maakond          32
Võru maakond           28
Jõgeva maakond         25
Järva maakond          24
Põlva maakond          23
Lääne maakond          22
Saare maakond          18
Valga maakond          15
Hiiu maakond            9
Name: Maakond, dtype: int64

Asutused:


lasteaed                    301
põhikool                    196
gümnaasium                  124
lasteaed-põhikool            59
kutseõppeasutus              10
täiskasvanute gümnaasium      1
huvikool                      1
Name: Asutuse tüüp, dtype: int64

In [4]:
import re
    
alldevices = []
for i in range(len(data)):
    devicelist = str(data.iloc[i,4])
    devicesfromrow = re.split(";", devicelist)
    for device in devicesfromrow:
        if not device == "": 
            alldevices.append(device.strip())

alldevicesdf = pd.DataFrame(alldevices)
#alldevicesdf.head()
#alldevicesdf.value_counts()
pd.set_option('display.max_rows', alldevicesdf.shape[0]+1)
#alldevicesdf.sort_values(by=0)


In [5]:
def formatnumber(number):
    number=str(number).replace(',', '').replace('€', '').strip()
    return float(number)
    
#dictionary - county:(year:amount)

def getyearlyfundings(data):
    dictionary = {}
    
    #for rows
    for i in range(len(data)):
        county = data.iloc[i, 2]
        if county in dictionary:
            years = dictionary[county]
        else:
            years = {}
        
        #for each year
        for j in range(8):
            year = 2014+j
            cell = data.iloc[i, j+5]
            if pd.isna(cell):
                continue
            amount = formatnumber(cell)
            if year not in years: 
                years[year] = amount
            else:
                years[year] = years[year] + amount
            dictionary[county] = years
    return dictionary

fundings = getyearlyfundings(data)

for key, value in fundings.items():
    print(key, ' : ', value)

Viljandi maakond  :  {2014: 13870.39, 2019: 16946.89, 2015: 9910.32, 2016: 6012.16, 2021: 18631.350000000002, 2020: 13816.18, 2018: 17803.11, 2017: 2157.7200000000003}
Võru maakond  :  {2015: 10910.89, 2018: 17516.43, 2017: 14045.78, 2021: 11413.07, 2020: 9596.380000000001, 2019: 13667.509999999998, 2016: 5328.95, 2014: 4253.63}
Järva maakond  :  {2015: 16602.05, 2018: 4514.43, 2020: 12321.150000000001, 2021: 11130.300000000001, 2019: 22313.440000000002, 2016: 10041.9, 2014: 5019.51, 2017: 7603.039999999999}
Jõgeva maakond  :  {2015: 11958.859999999999, 2014: 8725.470000000001, 2018: 3371.13, 2021: 13602.7, 2016: 7121.92, 2020: 10971.81, 2017: 8397.15, 2019: 9661.59}
Harju maakond  :  {2014: 45922.229999999996, 2015: 72904.93, 2016: 78437.17000000001, 2019: 57712.649999999994, 2020: 105765.96999999999, 2021: 89713.94999999998, 2018: 65249.29000000001, 2017: 74521.35999999997}
Tartu maakond  :  {2014: 28784.46, 2021: 54010.67, 2016: 17320.18, 2018: 32653.899999999998, 2019: 57553.140000

In [10]:
for i in range(len(data)):
    if data.iloc[i, 0] == np.NaN:
        print(i)